In [2]:
import numpy as np; import pandas as pd; pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt; import seaborn as sns

train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

## EDA

In [3]:
train_df.head()

,Location,Cross_Street,Latitude,Longitude,Date_Reported,Date_Occurred,Time_Occurred,Area_ID,Area_Name,Reporting_District_no,Part 1-2,Modus_Operandi,Victim_Age,Victim_Sex,Victim_Descent,Premise_Code,Premise_Description,Weapon_Used_Code,Weapon_Description,Status,Status_Description,Crime_Category
0,4500 CARPENTER AV,NaN,34.1522,-118.3910,03/09/2020 12:00:00 AM,03/06/2020 12:00:00 AM,1800.0,15.0,N Hollywood,1563.0,1.0,0385,75.0,M,W,101.0,STREET,NaN,NaN,IC,Invest Cont,Property Crimes
1,45TH ST,ALAMEDA ST,34.0028,-118.2391,02/27/2020 12:00:00 AM,02/27/2020 12:00:00 AM,1345.0,13.0,Newton,1367.0,1.0,0906 0352 0371 0446 1822 0344 0416 0417,41.0,M,H,216.0,SWAP MEET,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont,Property Crimes
2,600 E MARTIN LUTHER KING JR BL,NaN,34.0111,-118.2653,08/21/2020 12:00:00 AM,08/21/2020 12:00:00 AM,605.0,13.0,Newton,1343.0,2.0,0329 1202,67.0,M,B,501.0,SINGLE FAMILY DWELLING,NaN,NaN,IC,Invest Cont,Property Crimes
3,14900 ORO GRANDE ST,NaN,34.2953,-118.4590,11/08/2020 12:00:00 AM,11/06/2020 12:00:00 AM,1800.0,19.0,Mission,1924.0,1.0,0329 1300,61.0,M,H,101.0,STREET,NaN,NaN,IC,Invest Cont,Property Crimes
4,7100 S VERMONT AV,NaN,33.9787,-118.2918,02/25/2020 12:00:00 AM,02/25/2020 12:00:00 AM,1130.0,12.0,77th Street,1245.0,1.0,0416 0945 1822 0400 0417 0344,0.0,X,X,401.0,MINI-MART,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont,Property Crimes


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Location               20000 non-null  object 
 1   Cross_Street           3448 non-null   object 
 2   Latitude               20000 non-null  float64
 3   Longitude              20000 non-null  float64
 4   Date_Reported          20000 non-null  object 
 5   Date_Occurred          20000 non-null  object 
 6   Time_Occurred          20000 non-null  float64
 7   Area_ID                20000 non-null  float64
 8   Area_Name              20000 non-null  object 
 9   Reporting_District_no  20000 non-null  float64
 10  Part 1-2               20000 non-null  float64
 11  Modus_Operandi         17259 non-null  object 
 12  Victim_Age             20000 non-null  float64
 13  Victim_Sex             17376 non-null  object 
 14  Victim_Descent         17376 non-null  object 
 15  Pr

In [5]:
train_df = train_df.drop_duplicates()
train_df.shape

(19987, 22)

In [6]:
train_df.isna().mean() * 100

Location                  0.000000
Cross_Street             82.753790
Latitude                  0.000000
Longitude                 0.000000
Date_Reported             0.000000
Date_Occurred             0.000000
Time_Occurred             0.000000
Area_ID                   0.000000
Area_Name                 0.000000
Reporting_District_no     0.000000
Part 1-2                  0.000000
Modus_Operandi           13.698904
Victim_Age                0.000000
Victim_Sex               13.118527
Victim_Descent           13.118527
Premise_Code              0.000000
Premise_Description       0.025016
Weapon_Used_Code         63.326162
Weapon_Description       63.326162
Status                    0.000000
Status_Description        0.000000
Crime_Category            0.000000
dtype: float64

Columns to drop: Cross_Street, Weapon_Description (Because more than 50% missing values)

## Preprocessing

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

In [8]:
train_df.drop_duplicates()
X = train_df.drop('Crime_Category',axis=1)
y = train_df['Crime_Category']
print(X.shape); X.head()

(19987, 21)


,Location,Cross_Street,Latitude,Longitude,Date_Reported,Date_Occurred,Time_Occurred,Area_ID,Area_Name,Reporting_District_no,Part 1-2,Modus_Operandi,Victim_Age,Victim_Sex,Victim_Descent,Premise_Code,Premise_Description,Weapon_Used_Code,Weapon_Description,Status,Status_Description
0,4500 CARPENTER AV,NaN,34.1522,-118.3910,03/09/2020 12:00:00 AM,03/06/2020 12:00:00 AM,1800.0,15.0,N Hollywood,1563.0,1.0,0385,75.0,M,W,101.0,STREET,NaN,NaN,IC,Invest Cont
1,45TH ST,ALAMEDA ST,34.0028,-118.2391,02/27/2020 12:00:00 AM,02/27/2020 12:00:00 AM,1345.0,13.0,Newton,1367.0,1.0,0906 0352 0371 0446 1822 0344 0416 0417,41.0,M,H,216.0,SWAP MEET,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont
2,600 E MARTIN LUTHER KING JR BL,NaN,34.0111,-118.2653,08/21/2020 12:00:00 AM,08/21/2020 12:00:00 AM,605.0,13.0,Newton,1343.0,2.0,0329 1202,67.0,M,B,501.0,SINGLE FAMILY DWELLING,NaN,NaN,IC,Invest Cont
3,14900 ORO GRANDE ST,NaN,34.2953,-118.4590,11/08/2020 12:00:00 AM,11/06/2020 12:00:00 AM,1800.0,19.0,Mission,1924.0,1.0,0329 1300,61.0,M,H,101.0,STREET,NaN,NaN,IC,Invest Cont
4,7100 S VERMONT AV,NaN,33.9787,-118.2918,02/25/2020 12:00:00 AM,02/25/2020 12:00:00 AM,1130.0,12.0,77th Street,1245.0,1.0,0416 0945 1822 0400 0417 0344,0.0,X,X,401.0,MINI-MART,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont


In [2]:
def extract_location_data(s):
    street_number = direction = street_name = street_type = None
    try: # steet number
        nt = s.split(' ')[0]
        street_number = nt if int(nt.isdigit()) else None
    except: pass
    try: # direction
        nt = [x.strip() for x in s.split(' ') if x.strip()]
        for x in nt:
            if x.isdigit(): continue
            elif len(x)>1: break
            if x in ['N', 'S', 'E', 'W']:
                direction = x; break
    except: pass
    try: # street type
        nt = s.split('   ')[-1].strip()
        street_type = nt if len(nt)==2 and not(nt.isdigit()) else None
    except: pass

    street_name = None # street name
    nt = [x.strip() for x in s.split('  ') if x.strip()]
    for x in nt:
        test_x = [x.strip() for x in x.split(' ') if x.strip()]
        if len(test_x)==1 and not(street_number and street_number == test_x[0]) and not(street_type and street_type == test_x[0]):
            street_name = x; break
        elif not(street_number and street_number in test_x):
            street_name = x; break
        
    return street_number, direction, street_name, street_type
def split_location_column(df, location_col='Location'):
    parsed_data = []
    for idx, row in df.iterrows():
        string = row[location_col]
        street_number, direction, street_name, street_type = extract_location_data(string)
        parsed_data.append({
            'Location': string,
            'Street_Number': street_number,
            'Direction': direction,
            'Street_Name': street_name,
            'Street_Type': street_type
        })
    location_df = pd.DataFrame(parsed_data, index=df.index)
    df = df.drop(columns=[location_col])
    df = pd.concat([df, location_df], axis=1)
    return location_df

In [10]:
X = split_location_column(X)
test_df = split_location_column(test_df)
print(X.shape, test_df.shape)
X.head()

(19987, 25) (5000, 25)


,Cross_Street,Latitude,Longitude,Date_Reported,Date_Occurred,Time_Occurred,Area_ID,Area_Name,Reporting_District_no,Part 1-2,Modus_Operandi,Victim_Age,Victim_Sex,Victim_Descent,Premise_Code,Premise_Description,Weapon_Used_Code,Weapon_Description,Status,Status_Description,Location,Street_Number,Direction,Street_Name,Street_Type
0,NaN,34.1522,-118.3910,03/09/2020 12:00:00 AM,03/06/2020 12:00:00 AM,1800.0,15.0,N Hollywood,1563.0,1.0,0385,75.0,M,W,101.0,STREET,NaN,NaN,IC,Invest Cont,4500 CARPENTER AV,4500,None,CARPENTER,AV
1,ALAMEDA ST,34.0028,-118.2391,02/27/2020 12:00:00 AM,02/27/2020 12:00:00 AM,1345.0,13.0,Newton,1367.0,1.0,0906 0352 0371 0446 1822 0344 0416 0417,41.0,M,H,216.0,SWAP MEET,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont,45TH ST,None,None,45TH,ST
2,NaN,34.0111,-118.2653,08/21/2020 12:00:00 AM,08/21/2020 12:00:00 AM,605.0,13.0,Newton,1343.0,2.0,0329 1202,67.0,M,B,501.0,SINGLE FAMILY DWELLING,NaN,NaN,IC,Invest Cont,600 E MARTIN LUTHER KING JR BL,600,E,MARTIN LUTHER KING JR,BL
3,NaN,34.2953,-118.4590,11/08/2020 12:00:00 AM,11/06/2020 12:00:00 AM,1800.0,19.0,Mission,1924.0,1.0,0329 1300,61.0,M,H,101.0,STREET,NaN,NaN,IC,Invest Cont,14900 ORO GRANDE ST,14900,None,ORO GRANDE,ST
4,NaN,33.9787,-118.2918,02/25/2020 12:00:00 AM,02/25/2020 12:00:00 AM,1130.0,12.0,77th Street,1245.0,1.0,0416 0945 1822 0400 0417 0344,0.0,X,X,401.0,MINI-MART,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont,7100 S VERMONT AV,7100,S,VERMONT,AV


Converting Modus textual data to numerical data

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X['Modus_Operandi'] = X['Modus_Operandi'].fillna('')

print(X['Modus_Operandi'].shape)

vectorizer.fit(X['Modus_Operandi'])
modus = vectorizer.transform(X['Modus_Operandi'])
feature_names = vectorizer.get_feature_names_out()

print(modus.shape)

modus_data = pd.DataFrame(data=modus.toarray(), columns=feature_names)

(19987,)
(19987, 481)


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X['Location'] = X['Location'].fillna('')

print(X['Location'].shape)

vectorizer.fit(X['Location'])
location = vectorizer.transform(X['Location'])
feature_names = vectorizer.get_feature_names_out()

print(location.shape)

location_data = pd.DataFrame(data=location.toarray(), columns=feature_names)

(19987,)
(19987, 3060)


In [27]:
processed_data = pd.concat([location_data,modus_data],axis=1)
print(processed_data.shape)
processed_data.head()

(19987, 3541)


00  00000  00100  01200  01500  02200  02900  04100  05000  05900  09100  \
0  0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1  0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2  0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3  0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4  0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

   100  1000  10000  100th  10100  101st  10200  102nd  10300  103rd  10400  \
0  0.0   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1  0.0   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2  0.0   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3  0.0   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4  0.0   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

   104th  10500  105th  10600  106th  10700  107th  10800  108th  10900  \
0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

   109th  10th  1100  11000  110th  11100  111th  11200  112th  11300  113th  \
0    0.0   0.0   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1    0.0   0.0   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2    0.0   0.0   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3    0.0   0.0   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4    0.0   0.0   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

   11400  114th  11500  115th  11600  116th  11700  117th  11800  118th  \
0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

   11900  119th  11th  1200  12000  120th  12100  12200  12300  123rd  12400  \
0    0.0    0.0   0.0   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1    0.0    0.0   0.0   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2    0.0    0.0   0.0   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3    0.0    0.0   0.0   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4    0.0    0.0   0.0   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

   12500  12600  126th  12700  127th  12800  12900  129th  12th  1300  13000  \
0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   0.0   0.0    0.0   
1    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   0.0   0.0    0.0   
2    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   0.0   0.0    0.0   
3    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   0.0   0.0    0.0   
4    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   0.0   0.0    0.0   

   13100  13200  132nd  13300  133rd  13400  134th  13500  13600  13700  \
0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

   13800  13900  13th  1400  14000  14100  14200  14300  14400  14500  145th  \
0    0.0    0.0   0.0   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1    0.0    0.0   0.0   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2    0.0    

### Pipeline

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier

modus_operandi_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='')),
    ('tfidf', TfidfVectorizer())
])
extract_location_column = FunctionTransformer(split_location_column)
location_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='')),
    ('tfidf', extract_location_column)
])
preprocessor = ColumnTransformer(
    transformers=[
        ('modus_operandi', modus_operandi_transformer, ['Modus_Operandi']),
        ('location', location_transformer, ['Location'])
    ],
    # remainder='passthrough'  # To keep the other columns as is
)
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    # ('classifier', RandomForestClassifier())
])
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('modus_operandi',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='',
                                                                                 strategy='constant')),
                                                                  ('tfidf',
                                                                   TfidfVectorizer())]),
                                                  ['Modus_Operandi']),
                                                 ('location',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='',
                                                                                 strategy='constant')),
                                                                  ('tfidf',
                                                                   FunctionTransformer(func=<function split_location_column at 0x0000023071601300>))]),
                                                  ['Location'])]))])

In [36]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline.fit(X_train)
X_train_trans = pipeline.transform(X_train)
X_val_trans = pipeline.transform(X_val)

AttributeError: 'numpy.ndarray' object has no attribute 'lower'

## Training

In [316]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import accuracy_score

In [317]:
models = {
    'RandomForest': RandomForestClassifier(),
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'SVC': SVC(),
    'KNN': KNeighborsClassifier(),
    'NaiveBayes': GaussianNB(), #
    'DecisionTreeClassifier': DecisionTreeClassifier(),
    'GradientBoostingClassifier': GradientBoostingClassifier(),
}
best_model = None; best_accuracy = 0; best_model_name = ""
for model_name, model in models.items():
    model.fit(X_train_trans, y_train)
    val_predictions = model.predict(X_val_trans)
    accuracy = accuracy_score(y_val, val_predictions)
    print(f"Validation Accuracy for {model_name}: {accuracy}\n")
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model
        best_model_name = model_name
print(f"Best Model: {best_model_name}")
print(f"Best Model Accuracy: {best_accuracy}")

ValueError: Input X contains NaN.
SVC does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
# # reporting distric no as label
# Best Model Accuracy: 0.7965
# # reporting distric no as number
# Best Model Accuracy: 0.79525

# # part 1-2 as label
# Best Model Accuracy: 0.79525
# # part 1-2 as number
# 